In [1]:
%pip install memory_profiler

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


# Library Import

In [2]:
import pandas as pd
from memory_profiler import memory_usage
import time
import random

# Data import
https://www.kaggle.com/datasets/maharshipandya/-spotify-tracks-dataset

In [3]:
# Load the dataset
df = pd.read_csv('music_data.csv')

# Select only the relevant columns
filtered_df = df[['track_id', 'artists', 'album_name', 'track_name', 'popularity', 'track_genre']]


In [4]:
# Shuffle dataset function
def shuffle_dataset(data):
    data = data.sample(frac=1, random_state=random.randint(1, 2000)).reset_index(drop=True)
    return data

# Merge Sort

In [5]:
# Merge Sort implementation for `track_genre` and `popularity`
def merge_sort(data, key1, key2):
    """
    Merge Sort implementation for sorting by two keys:
    - Primary Key: `key1` ('track_genre') in ascending order.
    - Secondary Key: `key2` ('popularity') in descending order within key1 groups.
    
    Args:
        data (list): List of dictionaries to be sorted.
        key1 (str): Primary sorting key ('track_genre').
        key2 (str): Secondary sorting key ('popularity').

    Returns:
        list: Sorted list of dictionaries.
    """
    if len(data) <= 1:
        return data

    mid = len(data) // 2
    left = merge_sort(data[:mid], key1, key2)
    right = merge_sort(data[mid:], key1, key2)

    return merge_genre_popularity(left, right, key1, key2)

def merge_genre_popularity(left, right, key1, key2):
    """
    Merge function for sorting by two keys.

    Args:
        left (list): Left half of the data.
        right (list): Right half of the data.

    Returns:
        list: Merged and sorted list.
    """
    sorted_data = []
    i = j = 0

    while i < len(left) and j < len(right):
        # Compare by primary key (track_genre)
        if left[i][key1] < right[j][key1]:
            sorted_data.append(left[i])
            i += 1
        elif left[i][key1] > right[j][key1]:
            sorted_data.append(right[j])
            j += 1
        else:
            # If primary key is equal, compare by secondary key (popularity, descending)
            if left[i][key2] >= right[j][key2]:
                sorted_data.append(left[i])
                i += 1
            else:
                sorted_data.append(right[j])
                j += 1

    # Append remaining elements
    sorted_data.extend(left[i:])
    sorted_data.extend(right[j:])

    return sorted_data


# Quick Sort

In [6]:
def quick_sort(data, key1, key2, low=0, high=None):
    """
    Optimized Quick Sort with in-place partitioning and median-of-three pivot selection.
    Sorts data by key1 (genre) in ascending order and key2 (popularity) in descending order within key1 groups.

    Arguments explantion:
        data (list): List of dictionaries to be sorted.
        key1 (str): First key for sorting ('track_genre')
        key2 (str): Second key for sorting ('popularity')
        low (int): Starting index for the partition
        high (int): Ending index for the partition

    Returns:
        None: The list is sorted in place.
    """
    if high is None:
        high = len(data) - 1

    if low < high:
        # Get the partition index
        pivot_index = partition(data, key1, key2, low, high)

        # Recursively sort left and right partitions
        quick_sort(data, key1, key2, low, pivot_index - 1)
        quick_sort(data, key1, key2, pivot_index + 1, high)


def median_of_three(data, key1, key2, low, high):
    """
    Select the median-of-three pivot for better partitioning.
    Returns: int: Index of the median element.
    """
    mid = (low + high) // 2

    # Sort the low, mid, and high indices based on key1 and key2
    candidates = [low, mid, high]
    candidates.sort(key=lambda x: (data[x][key1], -data[x][key2]))  # Sort by key1, then key2 descending

    # Return the index of the median
    return candidates[1]


def partition(data, key1, key2, low, high):
    pivot_index = median_of_three(data, key1, key2, low, high)
    data[pivot_index], data[high] = data[high], data[pivot_index]
    pivot = data[high]
    i = low - 1
    for j in range(low, high):
        if data[j][key1] < pivot[key1] or (data[j][key1] == pivot[key1] and data[j][key2] >= pivot[key2]):
            i += 1
            data[i], data[j] = data[j], data[i]
    data[i + 1], data[high] = data[high], data[i + 1]
    return i + 1


# Benchmark

## Benchmark for quick sort

In [7]:
# Benchmarking function for Quick Sort
def benchmark_quick_sort(sort_function, sort_name):
    total_time = 0
    total_memory = 0
    runs = 5  # Number of runs
    sorted_data = None

    for _ in range(runs):
        # Shuffle the dataset
        shuffled_dataset = shuffle_dataset(filtered_df)
        data_list = shuffled_dataset.to_dict('records')

        # Measure memory usage and execution time
        mem_usage = memory_usage((sort_function, (data_list, 'track_genre', 'popularity')))
        start_time = time.time()
        sort_function(data_list, 'track_genre', 'popularity')
        end_time = time.time()

        # Calculate time and memory used
        time_used = end_time - start_time
        memory_used = max(mem_usage) - min(mem_usage)

        # Add to totals
        total_time += time_used
        total_memory += memory_used

        print(f"{sort_name} {_+1} attempt Execution Time: {time_used:.6f} seconds")
        print(f"{sort_name} {_+1} attempt Peak Memory Usage: {memory_used:.2f} MB")

        # Save the sorted data from the last run
        if _ == runs - 1:
            sorted_data = data_list

    # Calculate averages
    avg_time = total_time / runs
    avg_memory = total_memory / runs

    print(f"{sort_name} Average Execution Time: {avg_time:.6f} seconds")
    print(f"{sort_name} Average Peak Memory Usage: {avg_memory:.2f} MB")

    return sorted_data

# Run Quick Sort and benchmark it
print("\nBenchmarking Quick Sort:")
sorted_data_quick = benchmark_quick_sort(quick_sort, "Quick Sort")

# Display the sorted dataset
print("\nSorted Dataset (Quick Sort):")
sorted_df_quick = pd.DataFrame(sorted_data_quick)
print(sorted_df_quick.head())


Benchmarking Quick Sort:
Quick Sort 1 attempt Execution Time: 2.483814 seconds
Quick Sort 1 attempt Peak Memory Usage: 0.95 MB
Quick Sort 2 attempt Execution Time: 2.403848 seconds
Quick Sort 2 attempt Peak Memory Usage: 0.14 MB
Quick Sort 3 attempt Execution Time: 2.424670 seconds
Quick Sort 3 attempt Peak Memory Usage: 0.16 MB
Quick Sort 4 attempt Execution Time: 2.457765 seconds
Quick Sort 4 attempt Peak Memory Usage: 0.07 MB
Quick Sort 5 attempt Execution Time: 2.440058 seconds
Quick Sort 5 attempt Peak Memory Usage: 0.14 MB
Quick Sort Average Execution Time: 2.442031 seconds
Quick Sort Average Peak Memory Usage: 0.29 MB

Sorted Dataset (Quick Sort):
                 track_id                    artists  \
0  5vjLSffimiIP26QG5WcN2K           Chord Overstreet   
1  1EzrEOXmMH3G43AXT1y7pA                 Jason Mraz   
2  3S0OXQeoh0w6AY8WQVckRW                 Jason Mraz   
3  08MFgEQeVLF37EyZ7jcwLc               Zack Tabudlo   
4  0IktbUcnAGrvD03AWnz3Q8  Jason Mraz;Colbie Caillat   


## Benchmark for merge sort

In [8]:
# Benchmarking function for Merge Sort
def benchmark_merge_sort(sort_function, sort_name):
    total_time = 0
    total_memory = 0
    runs = 5  # Number of runs
    sorted_data = None

    for _ in range(runs):
        # Shuffle the dataset
        shuffled_dataset = shuffle_dataset(filtered_df)
        data_list = shuffled_dataset.to_dict('records')

        # Measure memory usage and execution time
        mem_usage = memory_usage((sort_function, (data_list, 'track_genre', 'popularity')))
        start_time = time.time()
        sorted_data = sort_function(data_list, 'track_genre', 'popularity')
        end_time = time.time()
        # Calculate time and memory used
        time_used = end_time - start_time
        memory_used = max(mem_usage) - min(mem_usage)

        # Add to totals
        total_time += time_used
        total_memory += memory_used

        print(f"{sort_name} {_+1} attempt Execution Time: {time_used:.6f} seconds")
        print(f"{sort_name} {_+1} attempt Peak Memory Usage: {memory_used:.2f} MB")

        # 
    # Calculate averages
    avg_time = total_time / runs
    avg_memory = total_memory / runs

    print(f"{sort_name} Average Execution Time: {avg_time:.6f} seconds")
    print(f"{sort_name} Average Peak Memory Usage: {avg_memory:.2f} MB")

    return sorted_data

# Run Merge Sort and benchmark it
print("\nBenchmarking Merge Sort:")
sorted_data_merge = benchmark_merge_sort(merge_sort, "Merge Sort")

# Display the sorted dataset
print("\nSorted Dataset (Merge Sort):")
sorted_df_merge = pd.DataFrame(sorted_data_merge)
print(sorted_df_merge.head())


Benchmarking Merge Sort:
Merge Sort 1 attempt Execution Time: 0.944921 seconds
Merge Sort 1 attempt Peak Memory Usage: 3.04 MB
Merge Sort 2 attempt Execution Time: 0.981021 seconds
Merge Sort 2 attempt Peak Memory Usage: 4.48 MB
Merge Sort 3 attempt Execution Time: 0.969810 seconds
Merge Sort 3 attempt Peak Memory Usage: 2.23 MB
Merge Sort 4 attempt Execution Time: 0.963612 seconds
Merge Sort 4 attempt Peak Memory Usage: 1.90 MB
Merge Sort 5 attempt Execution Time: 0.917795 seconds
Merge Sort 5 attempt Peak Memory Usage: 1.72 MB
Merge Sort Average Execution Time: 0.955432 seconds
Merge Sort Average Peak Memory Usage: 2.67 MB

Sorted Dataset (Merge Sort):
                 track_id                    artists  \
0  5vjLSffimiIP26QG5WcN2K           Chord Overstreet   
1  1EzrEOXmMH3G43AXT1y7pA                 Jason Mraz   
2  3S0OXQeoh0w6AY8WQVckRW                 Jason Mraz   
3  08MFgEQeVLF37EyZ7jcwLc               Zack Tabudlo   
4  0IktbUcnAGrvD03AWnz3Q8  Jason Mraz;Colbie Caillat   
